In [1]:
import pickle as pkl
import pandas as pd
import os
import numpy as np
from collections import Counter
import json

with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load both dataset and the set difference of previous selected works, then 

In [2]:
# os.chdir('..')

In [3]:
local_data = U.load_file('data_vLocal.pkl', 'pkl', config['DATADIR'])
# final_data = U.load_file('data_vFinal.pkl', 'pkl', config['DATADIR'])
ff_data = U.load_file('data_vFF.pkl', 'pkl', config['DATADIR'])
metadata = U.load_file('metadata.csv', 'csv', config['DATADIR'])

In [4]:
local_df = pd.DataFrame(local_data)
# final_df = pd.DataFrame(final_data)
ff_df = pd.DataFrame(ff_data)


Unique works for each of the three datasets:

In [5]:

len(local_df.book_id.unique())
# len(final_df.book_id.unique())
len(ff_df.book_id.unique())


6424

Because the final_df has a lot of text from non-English languages, I have decided to proceed as follows:
* Our overall dataset will be the `ff_df` object, corresponding to the `data_vFF.pkl` file. The filtering for this file was the most stringent.
* We will use the works not included in `ff_df`, but in `local_df`, as the training corpus for our word embeddings

In [6]:
local_df['str_text_lines'] = local_df['text_lines'].apply(str)
local_df['passage_key'] = local_df['gutenbergbookid'] + '_' + local_df['str_text_lines']

ff_df['str_text_lines'] = ff_df['text_lines'].apply(str)
ff_df['passage_key'] = ff_df['gutenbergbookid'] + '_' + ff_df['str_text_lines']


In [7]:
dataset_passages = list(ff_df.passage_key.unique())

In [8]:
ldf_index = local_df.set_index('passage_key').index
ff_index = ff_df.set_index('passage_key').index
corpus_index = ldf_index.difference(ff_index)

In [9]:
local_df_idx = local_df.set_index('passage_key')
corpus_df = local_df_idx.loc[corpus_index].reset_index()

In [10]:
corpus_df['joined_text'] = corpus_df.text.apply(' '.join)

In [11]:
corpus_text = " ".join(corpus_df.joined_text)

In [12]:
with open('corpus_text.txt', 'w') as f:
    f.write(corpus_text)

In [ ]:
corpus_text